In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
datasets = []
for i in range(1, 5):
    d = np.load(f"dataset{i}.npy")
    if d.shape == (2, 10):
        d = d.T
    datasets.append(d)

In [ ]:
P = len(datasets)
N = datasets[0].shape[0]

In [ ]:
#a_p e b_p
A_list, b_list = [], []
for data in datasets:
    Ap = np.hstack([np.ones((N, 1)), -2 * data])
    bp = -np.sum(data**2, axis=1, keepdims=True)
    A_list.append(Ap)
    b_list.append(bp)

In [ ]:
#parameters do alm e bcd
c = 100
eps_ALM = 0.01
eps_BCD = 0.01

In [ ]:
x0 = np.zeros((3, 1))
xp = [np.zeros((3, 1)) for _ in range(P)]
lambdas = [np.zeros((3, 1)) for _ in range(P)]

In [ ]:
max_violation_hist = []
bcd_iters_per_alm = []

In [ ]:
#funcs auxiliares
def solve_local_agent(Ap, bp, x0_prev, lambdap, c):
    lhs = 2 * Ap.T @ Ap + c * np.eye(3)
    rhs = 2 * Ap.T @ bp + lambdap + c * x0_prev
    return np.linalg.solve(lhs, rhs)

In [ ]:
def solve_server(u_list, lambdas, c):
    P = len(u_list)
    term = np.sum([c * u_list[p] - lambdas[p] for p in range(P)], axis=0)
    return term / (P * c)

In [ ]:
def compute_violation(x0, xp):
    norms = [np.linalg.norm(x0 - xpi) / (np.linalg.norm(x0) + 1e-12) for xpi in xp]
    return max(norms)

In [ ]:
#loop principal do alm
max_ALM_iters = 50
for k in range(1, max_ALM_iters + 1):
    u0_prev = x0.copy()
    m = 1
    while True:
        u_list = [solve_local_agent(A_list[p], b_list[p], u0_prev, lambdas[p], c) for p in range(P)]
        u0_new = solve_server(u_list, lambdas, c)
        if np.linalg.norm(u0_new - u0_prev) <= eps_BCD:
            break
        u0_prev = u0_new
        m += 1
    x0 = u0_new
    xp = u_list
    bcd_iters_per_alm.append(m)
    for p in range(P):
        lambdas[p] = lambdas[p] + c * (x0 - xp[p])
    violation = compute_violation(x0, xp)
    max_violation_hist.append(violation)
    if violation <= eps_ALM:
        break

In [ ]:
#final results
y, cx, cy = x0.flatten()
cFL = np.array([cx, cy])
RFL = np.sqrt(np.linalg.norm(cFL)**2 - y)

In [ ]:
print(f"Centro federado: c_FL = ({cFL[0]:.2f}, {cFL[1]:.2f})")
print(f"Raio federado:   R_FL = {RFL:.2f}")
print(f"Nº iterações ALM: {k}")
print(f"BCD por ALM: {bcd_iters_per_alm}")

In [ ]:
theta = np.linspace(0, 2 * np.pi, 200)

In [ ]:
plt.figure(figsize=(8, 6))
colors = ["tab:blue", "tab:orange", "tab:green", "tab:red"]

In [ ]:
for p, data in enumerate(datasets):
    plt.scatter(data[:, 0], data[:, 1], color=colors[p], s=25, label=f"Dataset of agent {p+1}")
    A_loc, b_loc = A_list[p], b_list[p]
    x_ls, *_ = np.linalg.lstsq(A_loc, b_loc, rcond=None)
    y_ls, cx_ls, cy_ls = x_ls.flatten()
    R_ls = np.sqrt(np.linalg.norm([cx_ls, cy_ls])**2 - y_ls)
    x_circ = cx_ls + R_ls * np.cos(theta)
    y_circ = cy_ls + R_ls * np.sin(theta)
    plt.plot(x_circ, y_circ, linestyle="--", color=colors[p], alpha=0.7)

In [ ]:
x_circ_FL = cFL[0] + RFL * np.cos(theta)
y_circ_FL = cFL[1] + RFL * np.sin(theta)
plt.plot(x_circ_FL, y_circ_FL, "k", linewidth=2, label="FL fitted circle")

In [ ]:
plt.xlabel("x")
plt.ylabel("y")
plt.title("The four local datasets and the corresponding four local circles\nand the FL fitted circle")
plt.legend()
plt.axis("equal")
plt.grid(True)
plt.tight_layout()
plt.savefig(r"C:\Users\User1\Downloads\task11_circles.pdf", bbox_inches="tight", format="pdf")
plt.show()

In [ ]:
plt.figure()
ks = range(1, len(max_violation_hist) + 1)
plt.semilogy(ks, max_violation_hist, marker="o")
plt.xlabel("ALM iteration (k)")
plt.ylabel("max(||x0 - xp|| / ||x0||) over agents")
plt.title("Maximum constraint violation across ALM iterations")
plt.grid(True, which="both")
plt.tight_layout()
plt.savefig(r"C:\Users\User1\Downloads\task11_convergence.pdf", bbox_inches="tight", format="pdf")
plt.show()